In [1]:
import sqlite3
import pandas as pd
import sys
import numpy as np
import argparse
from os import listdir
from os.path import isfile, join

class args:
    i = "/Users/seunfuta/Downloads/NIST/IMG/sdelete-W7x64.db" #Wireshark-W7x64.db"
    c = "/Users/seunfuta/Downloads/NIST/OluDB_combo_v3.db"
    o = "/Users/seunfuta/Downloads/NIST/OLUSCAN/"
if __name__ == '__main__':
    #parser = argparse.ArgumentParser(description='Olu method')
    #parser.add_argument('-c', action="store", default="/Users/seunfuta/Downloads/NIST/OluDB_combo_v3.db", help='catalog path')
    #parser.add_argument('-i', action="store", default="/Users/seunfuta/Downloads/NIST/IMG/Wireshark-W7x64.db", help="image path folder")
    #parser.add_argument('-o', action="store", default="/Users/seunfuta/Downloads/NIST/OLUSCAN/", help='output csv')
    #args = parser.parse_args()
    
    #onlyfiles = [f for f in listdir(args.i) if isfile(join(args.i, f))]
    #for file in onlyfiles:
    #    print("IMAGE "+str(onlyfiles.index(file)) +" out of "+str(len(onlyfiles)))
    #    main(join(args.i+file))
    #main(join(args.i))
    #def main(imagefilepath):
    CATALOG_DB_PATH = args.c
    catalog_conn = sqlite3.connect(CATALOG_DB_PATH)
    catalog_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize','app','app_id'])
    catalog_df = pd.read_sql_query("SELECT block_hashes.obj_id, block_hashes.inode, block_hashes.filename, block_hashes.file_offset, \
                    block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize , files.app, files.app_id\
                    FROM files \
                    INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id \
                    and files.inode = block_hashes.inode and files.filename=block_hashes.filename;", catalog_conn)
    print("original length ",len(catalog_df))
    catalog_df = catalog_df[catalog_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
    catalog_df = catalog_df[catalog_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
    catalog_df = catalog_df[catalog_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
    catalog_df = catalog_df[catalog_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
    catalog_conn.close()
    print("catalog app length, ", len(catalog_df))
    #############
    print(args.i)#magefilepath)
    IMAGE_DB_PATH = args.i#imagefilepath
    image_conn = sqlite3.connect(IMAGE_DB_PATH)
    image_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize'])
    image_df = pd.read_sql_query("SELECT block_hashes.obj_id, files.inode, files.filename, block_hashes.file_offset, \
                    block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize \
                    FROM files \
                    INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id;", image_conn)
    print("original image length ",len(image_df))
    image_df = image_df[image_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
    image_df = image_df[image_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
    image_df = image_df[image_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
    image_df = image_df[image_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
    print("current image length ",len(image_df))
    ##############


original length  6826014
catalog app length,  6546710
/Users/seunfuta/Downloads/NIST/IMG/sdelete-W7x64.db
original image length  6415822
current image length  875101


In [2]:
    from collections import OrderedDict
    app_list = catalog_df.app.unique() #['Wireshark-W7x64'] #
    result_df = pd.DataFrame()
    for app in ['TrueCrypt63-WinXP']:#sdelete-W7x64']:#app_list:
        #if app == 'OfficePro2003-W7x32':
        app_df = catalog_df[catalog_df.app == app]
        app_unique_md5s= app_df.md5.unique()
        matched_image_df = image_df[image_df.md5.isin(app_unique_md5s)]
        #print(app," matched number of sectors in image ",len(matched_image_df))
        #print(matched_image_df)
        #lets create app sec pairs
        app_files = app_df.filename.unique()
        #print(app_files)
        app_pairs_set = {} #dict #set() 
        #lst2 = list(matched_image_df.md5)
        #lst2_pairs = list(map(lambda a, b: a + b, lst2[:-1], lst2[1:]))
        #Prob_Total = float(0)
        file1 = app_files[2]
        print(file1)



Program Files/TrueCrypt/TrueCrypt Format.exe


In [5]:
app_file = app_df[app_df.filename == file1]
app_file

,obj_id,inode,filename,file_offset,len,md5,sha1,partition,filesize,app,app_id
912756,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,0,512,8448b5c2c50d8af13f838caae279eccb,caa78adb5871f679e82154063cbf6358e8881fb6,2,155736,sdelete-W7x64,9480-2-15142-1
912757,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,512,512,de2049a0d59bb1a19c240119b07389ba,b1e726f32a18b4cd391e1c12d0a7227fae97d6fb,2,155736,sdelete-W7x64,9480-2-15142-1
912758,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,1024,512,baeac4ec34b91b54be5c5adf891e3ec4,8d735aaac0e0fce02e3c1020556a8c31eba65bb3,2,155736,sdelete-W7x64,9480-2-15142-1
912759,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,1536,512,34e28a174f88c9a0d34217ea15fb2126,ba0994b7cc9d74acde566e598e613b981c109de2,2,155736,sdelete-W7x64,9480-2-15142-1
912760,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,2048,512,75de2ebc36a42091b9247a1afd1da711,d65fa8e813805d42613aed3389815621d76ac70b,2,155736,sdelete-W7x64,9480-2-15142-1
...,...,...,...,...,...,...,...,...,...,...,...
913056,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,153600,512,386ef1b1ef5998bc79765891a71d45cf,ec73e41513f9150cfc6c647d057690b1d0c709ee,2,155736,sdelete-W7x64,9480-2-15142-1
913057,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,154112,512,a93471e9f3114d7ff5635018d867941f,676cdbd19dd601246906b6d7361280ec7d40f161,2,155736,sdelete-W7x64,9480-2-15142-1
913058,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,154624,512,a25acc3f4732bc9f330415f517307987,d7affe06e6e31662a88b5441e06e8206c44d0c27,2,155736,sdelete-W7x64,9480-2-15142-1
913059,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,155136,512,6ed2b9f5a1c947ab20726f0de5b4c630,7bc2a7bbffafc1e4e61a3df6258a0a672582793e,2,155736,sdelete-W7x64,9480-2-15142-1


In [6]:
matched_image_df.head(4)

,obj_id,inode,filename,file_offset,len,md5,sha1,partition,filesize
656389,5453,58371,Users/nsrl-user/Desktop/sdelete.exe,0,512,8448b5c2c50d8af13f838caae279eccb,caa78adb5871f679e82154063cbf6358e8881fb6,2,155736
656390,5453,58371,Users/nsrl-user/Desktop/sdelete.exe,512,512,de2049a0d59bb1a19c240119b07389ba,b1e726f32a18b4cd391e1c12d0a7227fae97d6fb,2,155736
656391,5453,58371,Users/nsrl-user/Desktop/sdelete.exe,1024,512,baeac4ec34b91b54be5c5adf891e3ec4,8d735aaac0e0fce02e3c1020556a8c31eba65bb3,2,155736
656392,5453,58371,Users/nsrl-user/Desktop/sdelete.exe,1536,512,34e28a174f88c9a0d34217ea15fb2126,ba0994b7cc9d74acde566e598e613b981c109de2,2,155736


In [17]:
matched_image_df.to_csv("/Users/seunfuta/Downloads/matched_image_df.csv")

In [19]:
image_df[image_df.md5 == '8448b5c2c50d8af13f838caae279eccb']

,obj_id,inode,filename,file_offset,len,md5,sha1,partition,filesize
656389,5453,58371,Users/nsrl-user/Desktop/sdelete.exe,0,512,8448b5c2c50d8af13f838caae279eccb,caa78adb5871f679e82154063cbf6358e8881fb6,2,155736
1067447,5184,58345,Users/nsrl-user/AppData/Local/Temp/Temp1_SDele...,0,512,8448b5c2c50d8af13f838caae279eccb,caa78adb5871f679e82154063cbf6358e8881fb6,2,155736


In [20]:
imagex_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize'])
imagex_df = pd.read_sql_query("SELECT block_hashes.obj_id, files.inode, files.filename, block_hashes.file_offset, \
                    block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize \
                    FROM files \
                    INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id;", image_conn)
print("original image length ",len(imagex_df))
#imagex_df = imagex_df[imagex_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
#imagex_df = imagex_df[imagex_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
#imagex_df = imagex_df[imagex_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
#imagex_df = imagex_df[imagex_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
#print("current image length ",len(imagex_df))

original image length  6415822


In [4]:
image_df[image_df.md5 == '8448b5c2c50d8af13f838caae279eccb']

NameError: name 'image_df' is not defined

In [3]:
matched_image_df.loc[:,'image_offset'] = matched_image_df.index
###notthing
#matched_image_df_dict = matched_image_df.to_dict('records')
#series_filename = pd.Series()
#series_file_off = pd.Series()
appfilenames= app_df.filename.unique()
for filename in appfilenames:
    #print(filename)
    series_file_off = pd.Series(index=[0,1,2,3,4])
    appfile_df = app_df[app_df.filename == filename]
    matched_image_appfile_df = matched_image_df[matched_image_df.filename == filename]
    print(len(matched_image_appfile_df)," sectors", filename)
    matched_image_appfile_dict = matched_image_appfile_df.to_dict('records')
    counter = 0
    for row in matched_image_appfile_dict:
        #print(row)
        cluster_off = row['image_offset'] % 8
        
        catapphash_off = np.array(appfile_df.index[appfile_df.md5 == row['md5']])
        catapphash_off = catapphash_off % 8
        catapphash_filename = np.array(appfile_df.filename[appfile_df.md5 == row['md5']])
        catapphash_file_off = np.array(appfile_df.file_offset[appfile_df.md5 == row['md5']])/512
        #print(cluster_off, row['md5'],cluster_off in catapphash_off)
        #if cluster_off in catapphash_off:
        print("image", row['image_offset'], row['md5'], "cluster off", cluster_off, catapphash_filename, catapphash_file_off )
        #series_filename[len(series_filename)] = catapphash_filename
        series_file_off[counter]= catapphash_file_off
        counter +=1
    print(series_file_off)
    exit()
matched_image_df.loc[:,'match_appfile'] = series_filename
print("length series_filename", len(series_filename))
matched_image_df.loc[:,'match_appfile_off'] = series_file_off
print("length series_file_off", len(series_file_off))

/Users/seunfuta/opt/anaconda3/envs/paper2/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/var/folders/s7/d__51l693s13d_yt81j0qp9r0000gn/T/ipykernel_33061/362815207.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_file_off = pd.Series(index=[0,1,2,3,4])


0  sectors Program Files/TrueCrypt/TrueCrypt User Guide.pdf
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64
0  sectors Documents and Settings/All Users/Application Data/TrueCrypt
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64
0  sectors Program Files/TrueCrypt/TrueCrypt Format.exe
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64
0  sectors Program Files/TrueCrypt/truecrypt.sys
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64
0  sectors Program Files/TrueCrypt/TrueCrypt Setup.exe
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64
0  sectors WINDOWS/system32/drivers/truecrypt.sys
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64
0  sectors Documents and Settings/All Users/Start Menu/Programs/TrueCrypt/Uninstall TrueCrypt.lnk
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64
0  sectors WINDOWS/Prefetch/TRUECRYPT SETUP 6.3A.EXE-0A3B90AD.pf
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64
0  sectors Documents and Settings/All Users/Start Menu/P

NameError: name 'series_filename' is not defined

In [8]:
import pandas as pd
sa = pd.Series([11,12,13,14,151,16,17,18,21,22,23,24])
print(sa.index)
for elem_indx in range(len(sa)-1):
    elem = sa[elem_indx]
    next_elem = sa[elem_indx+1]
    print()

RangeIndex(start=0, stop=12, step=1)
0
1
2
3
4
5
6
7
8
9
10
11


In [49]:
import pandas as pd
sa = pd.Series([11,12,13,14,151,16,17,18,21,22,23,24])
lst2_pairs_f = list(map(lambda a, b: 1 if b-a==1 else 0, sa[:-1], sa[1:]))
lst2_pairs_b = list(map(lambda a, b: 1 if a-b==1 else 0, sa[1:],sa[:-1]))
if lst2_pairs_f[-1] == 1: lst2_pairs_f.extend([1]) 
else: lst2_pairs_f.extend([0])
if lst2_pairs_b[0] == 1: lst2_pairs_b.insert (0, 1) 
else: lst2_pairs_b.insert (0, 0)  
print(lst2_pairs_f)
print(lst2_pairs_b)
lst2_pairs = (pd.Series(lst2_pairs_f) | pd.Series(lst2_pairs_b))
print(lst2_pairs)
print(lst2_pairs.sum())

[1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1]
0     1
1     1
2     1
3     1
4     0
5     1
6     1
7     1
8     1
9     1
10    1
11    1
dtype: int64
11


In [39]:
la = [11,12,13,14,151,16,17,18,21,22,23,24]
aa = [1]
la.insert (0, 1) 
print(la)

[1, 11, 12, 13, 14, 151, 16, 17, 18, 21, 22, 23, 24]


In [5]:
files_df = app_df[app_df.filename == file1]

In [7]:
files_df.reset_index(drop=True, inplace=True)
print(files_df)


      obj_id  inode                                      filename  \
0       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
1       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
2       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
4       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
...      ...    ...                                           ...   
3031    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3032    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3033    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3034    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3035    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   

      file_offset  len                               md5  \
0               0  512  f7f9fd15dc6b25615dced7f50e57922c   
1             512  512  b640f487d6a0d49506266a74f88

In [37]:
file_hashes = files_df.md5.unique()
matched_image_df['image_offset'] = matched_image_df.index
matched_appfile_image_df = matched_image_df[matched_image_df.md5.isin(file_hashes)]
print("orig. image: ",len(matched_image_df), " new image: ", len(matched_appfile_image_df))

orig. image:  85  new image:  66


/var/folders/s7/d__51l693s13d_yt81j0qp9r0000gn/T/ipykernel_769/654800188.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_image_df['image_offset'] = matched_image_df.index


In [38]:
print(matched_appfile_image_df.md5)

656673     b27431292abccda8fec17eb690b82eb8
1067731    b27431292abccda8fec17eb690b82eb8
5716000    4f73573c1b597d9e6ab273854880b6c7
5716001    e557020f519052b3b696ace0d10d4fef
5716337    4f73573c1b597d9e6ab273854880b6c7
                         ...               
6301887    e557020f519052b3b696ace0d10d4fef
6301904    4f73573c1b597d9e6ab273854880b6c7
6301905    e557020f519052b3b696ace0d10d4fef
6302272    4f73573c1b597d9e6ab273854880b6c7
6302273    e557020f519052b3b696ace0d10d4fef
Name: md5, Length: 66, dtype: object


In [39]:
cat_off = np.array(files_df.index[files_df.md5 == 'b27431292abccda8fec17eb690b82eb8'])
cat_off = cat_off % 8
print(cat_off)

[0]


In [40]:
matched_off = np.array(matched_appfile_image_df.index[matched_appfile_image_df.md5 == 'b27431292abccda8fec17eb690b82eb8'])
print(matched_off)
matched_off = matched_off % 8
print(matched_off)
print(type(matched_off))

[ 656673 1067731]
[1 3]
<class 'numpy.ndarray'>


In [34]:
matched_off in cat_off
file_hashes_index_dict = {}

False

In [34]:
def condition(row):
    #print(row)
    matched_off = np.array(matched_appfile_image_df.index[matched_appfile_image_df.md5 == row])
    print(matched_off)
    matched_off = matched_off % 8
    print(matched_off)
    cat_off = np.array(files_df.index[files_df.md5 == row])
    print(cat_off)
    cat_off = cat_off % 8
    print(cat_off)
    return matched_off in cat_off

result = np.where(condition(matched_appfile_image_df.md5), 1, 0)

[ 656673 1067731 5716000 5716001 5716337 5716338 5716485 5716486 5716633
 5716634 5716790 5716791 5716950 5716951 5717107 5717108 5717265 5717266
 5717424 5717425 5717581 5717582 5717732 5717733 5717882 5717883 5718039
 5718040 5718195 5718196 5718353 5718354 5718510 5718511 5719444 5719445
 5719800 5719801 5721583 5721584 5722270 5722271 5723330 5723331 5723465
 5723466 6299562 6299563 6300531 6300844 6300845 6301642 6301643 6301689
 6301690 6301708 6301709 6301819 6301868 6301869 6301886 6301887 6301904
 6301905 6302272 6302273]
[1 3 0 1 1 2 5 6 1 2 6 7 6 7 3 4 1 2 0 1 5 6 4 5 2 3 7 0 3 4 1 2 6 7 4 5 0
 1 7 0 6 7 2 3 1 2 2 3 3 4 5 2 3 1 2 4 5 3 4 5 6 7 0 1 0 1]


ValueError: Can only compare identically-labeled Series objects

In [47]:
#from tqdm import tqdm
#import time
df_dict = matched_appfile_image_df.to_dict('records')
for row in df_dict:
    cluster_off = row['image_offset'] % 8
    catappfilehash_off = np.array(files_df.index[files_df.md5 == row['md5']])
    catappfilehash_off = catappfilehash_off % 8
    print(cluster_off, row['md5'],cluster_off in catappfilehash_off)

1 b27431292abccda8fec17eb690b82eb8 False
3 b27431292abccda8fec17eb690b82eb8 False
0 4f73573c1b597d9e6ab273854880b6c7 False
1 e557020f519052b3b696ace0d10d4fef False
1 4f73573c1b597d9e6ab273854880b6c7 False
2 e557020f519052b3b696ace0d10d4fef False
5 4f73573c1b597d9e6ab273854880b6c7 False
6 e557020f519052b3b696ace0d10d4fef False
1 4f73573c1b597d9e6ab273854880b6c7 False
2 e557020f519052b3b696ace0d10d4fef False
6 4f73573c1b597d9e6ab273854880b6c7 True
7 e557020f519052b3b696ace0d10d4fef True
6 4f73573c1b597d9e6ab273854880b6c7 True
7 e557020f519052b3b696ace0d10d4fef True
3 4f73573c1b597d9e6ab273854880b6c7 False
4 e557020f519052b3b696ace0d10d4fef False
1 4f73573c1b597d9e6ab273854880b6c7 False
2 e557020f519052b3b696ace0d10d4fef False
0 4f73573c1b597d9e6ab273854880b6c7 False
1 e557020f519052b3b696ace0d10d4fef False
5 4f73573c1b597d9e6ab273854880b6c7 False
6 e557020f519052b3b696ace0d10d4fef False
4 4f73573c1b597d9e6ab273854880b6c7 False
5 e557020f519052b3b696ace0d10d4fef False
2 4f73573c1b597d9e6a

In [26]:
files_hashes = files_df.md5.unique()


def is_clusteroff_valid(hash,ind):
    if true:
        return True




result = np.where(is_clusteroff_valid(matched_appfile_image_df,files_df), 1, 0)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
        for file in list(file1):#app_files:
            #file_hash_dict = {}
            #file_hashpair_set = set()
            files_df = app_df[app_df.filename == file]
            #print(f'file {file} is of size {len(files_df)}')
            file_hashes = files_df.md5
            '''
            if len(file_hashes)< 2:
                 #print(file_hashes.iloc[0])
                file_hashpair_set.add(file_hashes.iloc[0])
            else:
                for i in range(0, len(file_hashes)-1):
                    #print(file_hashes.iloc[i])
                    #print(file_hashes.iloc[int(i+1)])
                    hash_pair = file_hashes.iloc[i]+file_hashes.iloc[i+1]
                #print(i, hash_pair)
                    file_hashpair_set.add(hash_pair)
            app_pairs_set[file] = file_hashpair_set 
            '''
            file_hash_uniq_list = files_df.md5.unique()
            hash2pos = {}
            #result = hash2pos[]
            image_2_file_df = image_df[image_df.md5.isin(file_hash_uniq_list)]
            for uniq_hash in file_hash_uniq_list:
                hash2pos[uniq_hash] = list(map(lambda x: file_hash_uniq_list.index(x), uniq_hash))
            #image_hashes_dict = OrderedDict()
            result = list(map(lambda a, b, c: np.where((b-a ==1) & (c-b==1),1,0), hash2pos[image_2_file_df[:-2]], hash2pos[image_2_file_df[1:-1]], hash2pos[image_2_file_df[2:]]))
            